# Enhanced AMP Prediction - Quick Start Demo

This notebook demonstrates the basic usage of the Enhanced Antimicrobial Peptide Prediction system using ESM-650M embeddings and ensemble deep learning.

## Overview

The AMP prediction system includes:
- **ESM-650M embeddings** for rich protein representation
- **Ensemble of 6 models**: CNN, LSTM, GRU, BiLSTM, BiCNN, Transformer
- **Multiple prediction tasks**: Classification and MIC regression
- **Sequence optimization** using EvoGradient

## Setup and Imports

In [ ]:
# Install required packages (if not already installed)
# !pip install torch transformers pandas numpy plotly streamlit

import sys
from pathlib import Path

# Add src to path
notebook_dir = Path.cwd()
src_dir = notebook_dir.parent.parent / "src"
sys.path.append(str(src_dir))

import pandas as pd
import numpy as np
import torch
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, HTML

# Import our modules
from src.embeddings import ESMSequenceEmbedding, ESMAminoAcidEmbedding
from app.utils import DemoPredictor, validate_sequence, load_example_data
from app.utils.visualization import (
    create_prediction_plot, 
    create_confidence_plot,
    create_sequence_length_distribution
)

print("✅ Setup complete!")

## 1. Single Sequence Prediction

Let's start by predicting the antimicrobial activity of a single peptide sequence.

In [ ]:
# Initialize the demo predictor
predictor = DemoPredictor()
print("🤖 Demo predictor initialized")

# Example antimicrobial peptide (Magainin-2)
amp_sequence = "GIGKFLHSAKKFGKAFVGEIMNS"
print(f"\n🧬 Testing sequence: {amp_sequence}")
print(f"📏 Length: {len(amp_sequence)} amino acids")

In [ ]:
# Validate the sequence
is_valid, error_msg = validate_sequence(amp_sequence)
print(f"Validation: {'✅ Valid' if is_valid else f'❌ Invalid - {error_msg}'}")

if is_valid:
    # Make prediction
    result = predictor.predict_single(amp_sequence)
    
    # Display results
    prediction = "🦠 Antimicrobial Peptide" if result['ensemble']['prediction'] == 1 else "🚫 Non-Antimicrobial"
    confidence = result['ensemble']['confidence']
    
    print(f"\n🔮 Prediction: {prediction}")
    print(f"🎯 Confidence: {confidence:.2%}")
    
    print("\n📊 Individual Model Results:")
    for model_name, model_result in result['individual'].items():
        pred_text = "AMP" if model_result['prediction'] == 1 else "Non-AMP"
        conf_text = f"{model_result['confidence']:.2%}"
        print(f"  {model_name:12}: {pred_text:8} ({conf_text})")

## 2. Batch Analysis

Now let's analyze multiple sequences at once using our example dataset.

In [ ]:
# Load example dataset
example_df = load_example_data()
print(f"📊 Loaded {len(example_df)} example sequences")

# Display the dataset
display(example_df)

In [ ]:
# Analyze all sequences
sequences = example_df['Sequence'].tolist()
print("🔄 Analyzing all sequences...")

batch_results = predictor.predict_batch(sequences)

# Extract predictions and confidences
predictions = [r['ensemble']['prediction'] for r in batch_results]
confidences = [r['ensemble']['confidence'] for r in batch_results]

# Create results dataframe
results_df = example_df.copy()
results_df['Predicted'] = ['AMP' if p == 1 else 'Non-AMP' for p in predictions]
results_df['Confidence'] = confidences
results_df['Correct'] = results_df['Known_Label'] == predictions

print(f"\n📈 Results Summary:")
print(f"  Total sequences: {len(results_df)}")
print(f"  Predicted AMPs: {sum(predictions)}")
print(f"  Predicted Non-AMPs: {len(predictions) - sum(predictions)}")
print(f"  Average confidence: {np.mean(confidences):.2%}")
print(f"  Accuracy: {results_df['Correct'].mean():.2%}")

display(results_df)

## 3. Visualization

Let's create some visualizations to better understand our predictions.

In [ ]:
# Prediction confidence plot
fig = create_prediction_plot(batch_results)
fig.show()

print("📊 Each point represents a sequence, colored by prediction (AMP/Non-AMP)")

In [ ]:
# Individual model confidence for the first sequence
first_result = batch_results[0]
first_sequence = sequences[0]

print(f"🔍 Detailed analysis for: {first_sequence}")

fig = create_confidence_plot(first_result)
fig.show()

print("📊 Bar chart shows confidence from each model, with ensemble average as horizontal line")

In [ ]:
# Sequence length distribution
fig = create_sequence_length_distribution(sequences)
fig.show()

print("📏 Distribution of sequence lengths in our example dataset")

## 4. Sequence Analysis

Let's analyze the composition and properties of our sequences.

In [ ]:
from app.utils.demo_utils import analyze_sequence_composition

# Analyze composition of a known AMP
amp_seq = "GIGKFLHSAKKFGKAFVGEIMNS"  # Magainin-2
composition = analyze_sequence_composition(amp_seq)

print(f"🧬 Composition analysis for {amp_seq}:")
print(f"  Length: {composition['length']} amino acids")
print(f"  Hydrophobic ratio: {composition['hydrophobic_ratio']:.2%}")
print(f"  Charged (positive): {composition['charged_positive_ratio']:.2%}")
print(f"  Charged (negative): {composition['charged_negative_ratio']:.2%}")
print(f"  Aromatic ratio: {composition['aromatic_ratio']:.2%}")
print(f"  Net charge: {composition['net_charge']}")
print(f"  Charge density: {composition['charge_density']:.3f}")

In [ ]:
# Compare AMP vs Non-AMP compositions
amp_sequences = [seq for seq, label in zip(sequences, example_df['Known_Label']) if label == 1]
non_amp_sequences = [seq for seq, label in zip(sequences, example_df['Known_Label']) if label == 0]

print("📊 Comparing AMP vs Non-AMP compositions:")

# Calculate average compositions
amp_compositions = [analyze_sequence_composition(seq) for seq in amp_sequences]
non_amp_compositions = [analyze_sequence_composition(seq) for seq in non_amp_sequences]

properties = ['hydrophobic_ratio', 'charged_positive_ratio', 'aromatic_ratio', 'charge_density']

for prop in properties:
    amp_avg = np.mean([comp[prop] for comp in amp_compositions])
    non_amp_avg = np.mean([comp[prop] for comp in non_amp_compositions])
    
    print(f"  {prop.replace('_', ' ').title()}:")
    print(f"    AMPs: {amp_avg:.3f}")
    print(f"    Non-AMPs: {non_amp_avg:.3f}")
    print(f"    Difference: {amp_avg - non_amp_avg:.3f}")
    print()

## 5. Model Uncertainty

Let's explore prediction uncertainty using bootstrap sampling.

In [ ]:
# Predict with uncertainty estimation
test_sequence = "GLFDIVKKVVGALCS"
uncertainty_result = predictor.predict_with_uncertainty(test_sequence, n_samples=50)

print(f"🧬 Uncertainty analysis for: {test_sequence}")
print(f"🔮 Prediction: {'AMP' if uncertainty_result['prediction'] == 1 else 'Non-AMP'}")
print(f"🎯 Confidence: {uncertainty_result['confidence']:.3f}")
print(f"🌊 Uncertainty (std): {uncertainty_result['uncertainty']:.3f}")
print(f"📊 95% Confidence Interval: [{uncertainty_result['confidence_interval']['lower']:.3f}, {uncertainty_result['confidence_interval']['upper']:.3f}]")

# Visualize uncertainty
conf_lower = uncertainty_result['confidence_interval']['lower']
conf_upper = uncertainty_result['confidence_interval']['upper']
confidence = uncertainty_result['confidence']

fig = go.Figure()

# Add confidence interval
fig.add_trace(go.Scatter(
    x=[test_sequence[:10] + '...'],
    y=[confidence],
    error_y=dict(
        type='data',
        symmetric=False,
        array=[conf_upper - confidence],
        arrayminus=[confidence - conf_lower]
    ),
    mode='markers',
    marker=dict(size=10, color='blue'),
    name='Prediction with Uncertainty'
))

fig.add_hline(y=0.5, line_dash="dash", line_color="gray", 
              annotation_text="Decision Threshold")

fig.update_layout(
    title="Prediction Uncertainty",
    yaxis_title="Confidence",
    yaxis=dict(range=[0, 1]),
    height=400
)

fig.show()

## 6. Sequence Variants

Let's explore how sequence mutations might affect predictions.

In [ ]:
from app.utils.demo_utils import generate_sequence_variants

# Generate variants of a known AMP
original_seq = "GIGKFLHSAKKFGKAFVGEIMNS"
variants = generate_sequence_variants(original_seq, n_variants=5)

print(f"🧬 Original sequence: {original_seq}")
print("🔬 Generated variants:")

# Predict for original and variants
all_sequences = [original_seq] + variants
all_results = predictor.predict_batch(all_sequences)

variant_df = pd.DataFrame({
    'Type': ['Original'] + [f'Variant {i+1}' for i in range(len(variants))],
    'Sequence': all_sequences,
    'Prediction': ['AMP' if r['ensemble']['prediction'] == 1 else 'Non-AMP' for r in all_results],
    'Confidence': [r['ensemble']['confidence'] for r in all_results]
})

display(variant_df)

# Visualize variant predictions
fig = px.bar(
    variant_df,
    x='Type',
    y='Confidence',
    color='Prediction',
    title="Prediction Confidence for Sequence Variants",
    color_discrete_map={'AMP': '#2E86C1', 'Non-AMP': '#E74C3C'}
)

fig.add_hline(y=0.5, line_dash="dash", line_color="gray")
fig.show()

print("📊 This shows how small mutations can affect AMP predictions")

## 7. Summary and Next Steps

This notebook demonstrated the basic functionality of the Enhanced AMP Prediction system. You've learned how to:

1. ✅ Make single sequence predictions
2. ✅ Perform batch analysis
3. ✅ Visualize results and model confidence
4. ✅ Analyze sequence composition
5. ✅ Estimate prediction uncertainty
6. ✅ Explore sequence variants

### Next Steps:

- **Explore the Streamlit app** for an interactive web interface
- **Check out other notebooks** for advanced features like:
  - ESM embedding generation
  - Model training and evaluation
  - EvoGradient optimization
  - Regression for MIC prediction

### Performance Highlights:

- **Classification Accuracy**: 93.57%
- **Precision**: 99.01% (very few false positives)
- **ROC-AUC**: 99.39% (excellent discrimination)
- **Ensemble of 6 models** for robust predictions

The system is ready for research and practical applications in antimicrobial peptide discovery! 🚀

In [ ]:
print("🎉 Demo completed successfully!")
print("🔗 Try the Streamlit app: streamlit run app/streamlit_app/app.py")
print("📚 Explore more notebooks in the app/notebooks/ directory")